# Conditional formatting based on other column values

## Compound cell values

In [ ]:
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer, VegaExpr

df = pd.DataFrame(
    {
        "column 1": [{"key": 11}, ["berry", "apple", "cherry"]],
        "column 2": [["berry", "berry", "cherry"], {"key": 10}],
    }
)

renderer = TextRenderer(
    background_color=VegaExpr(
        "cell.value[1] == 'berry' && cell.metadata.data['column 1']['key'] == 11 ? 'limegreen' : 'pink'"
    )
)

DataGrid(
    df,
    layout={"height": "100px"},
    base_column_size=150,
    default_renderer=renderer,
)

In [ ]:
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer, VegaExpr

df = pd.DataFrame(
    {
        "column 1": [{"key": {"nestedKey": 11}}, ["berry", "apple", "cherry"]],
        "column 2": [["berry", "berry", "cherry"], {"key": 10}],
    }
)

renderer = TextRenderer(
    background_color=VegaExpr(
        "cell.value[1] == 'berry' && \
    cell.metadata.data['column 1']['key']['nestedKey'] == 11 ? 'magenta' : 'dodgerblue'"
    )
)

DataGrid(
    df,
    layout={"height": "100px"},
    base_column_size=150,
    default_renderer=renderer,
)

In [ ]:
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer, VegaExpr

df = pd.DataFrame(
    {
        "column 1": [["one", ["two"]], ["berry", "apple", "cherry"]],
        "column 2": [["berry", "berry", "cherry"], ["one", ["two"]]],
    }
)

renderer = TextRenderer(
    background_color=VegaExpr(
        "cell.value[1] == 'berry' && \
    cell.metadata.data['column 1'][1][0] == 'two' ? 'pink' : 'teal'"
    )
)

DataGrid(
    df,
    layout={"height": "100px"},
    base_column_size=150,
    default_renderer=renderer,
)

In [ ]:
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer, VegaExpr

df = pd.DataFrame(
    {
        "column 1": [["one", ["two"]], ["berry", "apple", "cherry"]],
        "column 2": [["berry", "berry", "cherry"], ["one", ["two"]]],
    }
)

renderer = TextRenderer(
    background_color=VegaExpr("cell.value[1] == 'berry' ? 'pink' : 'teal'")
)

DataGrid(
    df,
    layout={"height": "100px"},
    base_column_size=150,
    default_renderer=renderer,
)

## Normal column names

In [ ]:
# Imports
import json
import numpy as np
import pandas as pd
from bqplot import DateScale, ColorScale
from py2vega.functions.type_coercing import toDate
from py2vega.functions.date_time import datetime
from ipydatagrid import Expr, DataGrid, TextRenderer

# Random data
n = 10
np.random.seed(0)
df = pd.DataFrame(
    {
        "Column 0": np.random.randn(n),
        "Column 1": np.random.randn(n),
        "Column 2": np.random.randn(n),
    }
)

# Formatting the values in column 1 based on the value of the silbing row in column 2
def format_based_on_other_column(cell):
    return "green" if cell.metadata.data["Column 2"] > 0.0 else "yellow"


column1_formatting = TextRenderer(
    text_color="black",
    background_color=Expr(format_based_on_other_column),
)

renderers = {
    "Column 1": column1_formatting,
}

grid = DataGrid(
    df,
    base_row_size=30,
    base_column_size=300,
    renderers=renderers,
    layout={"height": "350px"},
)
grid

## Example with nested columns

In [ ]:
import ipydatagrid as ipg
import pandas as pd
import numpy as np

top_level = [
    "Value Factors",
    "Value Factors",
    "Momentum Factors",
    "Momentum Factors",
]
bottom_level = ["Factor A", "Factor B", "Factor C", "Factor D"]

nested_df = pd.DataFrame(
    np.random.randn(4, 4).round(2),
    columns=pd.MultiIndex.from_arrays([top_level, bottom_level]),
    index=pd.Index(
        ["Security {}".format(x) for x in ["A", "B", "C", "D"]], name="Ticker"
    ),
)

# Formatting Factor B rows based on their siblings in the Factor C column
def format_based_on_other_column(cell):
    return (
        "green"
        if cell.value > -0
        and cell.metadata.data["('Value Factors', 'Factor B')"] > 0.0
        else "yellow"
    )


nested_grid = ipg.DataGrid(
    nested_df,
    base_column_size=90,
    layout={"height": "140px"},
    default_renderer=ipg.TextRenderer(
        horizontal_alignment="right",
        background_color=Expr(format_based_on_other_column),
    ),
)

nested_grid

In [ ]:
def format_based_on_other_column(cell):
    return (
        "green"
        if cell.column == 0 and cell.metadata.data["Signal"] == "Buy"
        else "red"
    )


signal_column_formatting = TextRenderer(
    text_color="white",
    background_color=Expr(format_based_on_other_column),
)

renderers = {
    "Stock": signal_column_formatting,
}

grid = DataGrid(
    pd.DataFrame(
        {"Stock": "A B C D".split(), "Signal": ["Buy", "Sell", "Buy", "Sell"]}
    ),
    base_row_size=30,
    base_column_size=300,
    renderers=renderers,
    layout={"height": "150px"},
)
grid

## Comparing dates

In [ ]:
import json

import numpy as np
import pandas as pd
from bqplot import DateScale, ColorScale
from py2vega.functions.type_coercing import toDate
from py2vega.functions.date_time import datetime

from ipydatagrid import Expr, DataGrid, TextRenderer, BarRenderer

n = 10
np.random.seed(0)


def format_based_on_date(cell):
    return (
        "magenta"
        if cell.column == 0 and cell.metadata.data["Dates"] >= "2020-10-21"
        else "yellow"
    )


df = pd.DataFrame(
    {
        "Value 1": np.random.randn(n),
        "Value 2": np.random.randn(n),
        "Dates": pd.date_range(end=pd.Timestamp("2020-10-25"), periods=n),
    }
)

text_renderer = TextRenderer(
    text_color="black", background_color=Expr(format_based_on_date)
)


def bar_color(cell):
    date = toDate(cell.value)
    return "green" if date > datetime("2000") else "red"


renderers = {
    "Value 1": text_renderer,
    "Dates": BarRenderer(
        bar_value=DateScale(min=df["Dates"][0], max=df["Dates"][n - 1]),
        bar_color=Expr(bar_color),
        format="%Y/%m/%d",
        format_type="time",
    ),
}

grid = DataGrid(
    df,
    base_row_size=30,
    base_column_size=300,
    renderers=renderers,
    layout={"height": "350px"},
)
grid